# Lab 2: Transformations vs Actions - Solutions

**Objective**: Master the fundamental distinction between transformations (lazy) and actions (eager) in Apache Spark.

**Learning Outcomes**:
- Understand lazy evaluation and when transformations are executed
- Identify transformations vs actions in Spark operations
- Observe execution timing and DAG construction
- Apply performance optimization techniques
- Debug execution plans and understand job stages

**Estimated Time**: 50 minutes

---

## Setup and Imports

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
import time
import pandas as pd

# Initialize Spark Session with detailed logging
spark = SparkSession.builder \
    .appName("Lab2-Transformations-vs-Actions") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")  # Suppress warnings for cleaner output

print(f"Spark version: {spark.version}")

# Enhanced Spark UI URL display
ui_url = spark.sparkContext.uiWebUrl
print(f"Spark UI: {ui_url}")
print("💡 In GitHub Codespaces: Check the 'PORTS' tab below for forwarded port 4040 to access Spark UI")

print(f"Application ID: {spark.sparkContext.applicationId}")

## Part 1: Understanding Transformations

Transformations are **lazy operations** that define a new RDD/DataFrame but don't compute results immediately.

### 1.1 Lazy Evaluation Demonstration

In [ ]:
# Load our transaction data
transactions_rdd = sc.textFile("../Datasets/customer_transactions.csv")
header = transactions_rdd.first()
transactions_no_header = transactions_rdd.filter(lambda line: line != header)

print("Data loaded. Let's observe lazy evaluation...")

In [ ]:
# Create a chain of transformations (all lazy!)
print("Creating transformation chain...")
start_time = time.time()

# Step 1: Parse transaction data
def parse_transaction(line):
    fields = line.split(',')
    return {
        'transaction_id': fields[0],
        'customer_id': fields[1],
        'amount': float(fields[2]),
        'category': fields[3],
        'payment_method': fields[4]
    }

parsed_transactions = transactions_no_header.map(parse_transaction)
print(f"✓ Map transformation defined - Time: {time.time() - start_time:.4f}s")

# Step 2: Filter for high-value transactions
high_value = parsed_transactions.filter(lambda t: t['amount'] > 100)
print(f"✓ Filter transformation defined - Time: {time.time() - start_time:.4f}s")

# Step 3: Extract amounts only
amounts_only = high_value.map(lambda t: t['amount'])
print(f"✓ Second map transformation defined - Time: {time.time() - start_time:.4f}s")

# Step 4: Create key-value pairs for category analysis
category_amounts = parsed_transactions.map(lambda t: (t['category'], t['amount']))
print(f"✓ Category mapping transformation defined - Time: {time.time() - start_time:.4f}s")

total_time = time.time() - start_time
print(f"\n🎯 All transformations defined in {total_time:.4f} seconds")
print("📝 Notice: No actual computation has happened yet!")

**Exercise 1.1**: Create your own transformation chain and measure definition time.

In [ ]:
# Solution: Create a transformation pipeline that:
# 1. Filters for Electronics category
# 2. Maps to extract customer_id and amount as tuple
# 3. Filters for amounts between $50-$500
# 4. Maps to create (customer_id, amount^2) pairs

print("Creating your transformation chain...")
start_time = time.time()

# Step 1: Filter for Electronics
electronics_only = parsed_transactions.filter(lambda t: t['category'] == 'Electronics')
print(f"✓ Electronics filter defined - Time: {time.time() - start_time:.4f}s")

# Step 2: Extract customer_id and amount
customer_amount_pairs = electronics_only.map(lambda t: (t['customer_id'], t['amount']))
print(f"✓ Customer-amount mapping defined - Time: {time.time() - start_time:.4f}s")

# Step 3: Filter for amount range $50-$500
amount_range_filtered = customer_amount_pairs.filter(lambda x: 50 <= x[1] <= 500)
print(f"✓ Amount range filter defined - Time: {time.time() - start_time:.4f}s")

# Step 4: Square the amounts
squared_amounts = amount_range_filtered.map(lambda x: (x[0], x[1] ** 2))
print(f"✓ Squared amounts mapping defined - Time: {time.time() - start_time:.4f}s")

total_time = time.time() - start_time
print(f"\n🎯 Your transformation chain defined in {total_time:.4f} seconds")

# Validation (this will be the first action!)
sample_result = squared_amounts.take(1)
assert len(sample_result) > 0, "Should have at least one result"
assert len(sample_result[0]) == 2, "Should be (customer_id, amount^2) pairs"
print("✓ Exercise 1.1 completed successfully!")
print(f"Sample result: {sample_result[0]}")

### 1.2 Examining the Execution Plan

In [ ]:
# Examine the lineage without triggering execution
print("Execution plan for amounts_only RDD:")
print(amounts_only.toDebugString().decode('utf-8'))

print("\n" + "="*50)
print("Number of partitions:", amounts_only.getNumPartitions())
print("Storage level:", amounts_only.getStorageLevel())

## Part 2: Understanding Actions

Actions are **eager operations** that trigger the execution of the entire transformation chain.

### 2.1 Triggering Execution with Actions

In [ ]:
# Now let's trigger execution with various actions
print("Triggering execution with actions...\n")

# Action 1: count()
print("1. Executing count() action...")
start_time = time.time()
count_result = amounts_only.count()
count_time = time.time() - start_time
print(f"   Result: {count_result} high-value transactions")
print(f"   Execution time: {count_time:.4f}s\n")

# Action 2: first()
print("2. Executing first() action...")
start_time = time.time()
first_result = amounts_only.first()
first_time = time.time() - start_time
print(f"   Result: ${first_result:.2f}")
print(f"   Execution time: {first_time:.4f}s\n")

# Action 3: take(5)
print("3. Executing take(5) action...")
start_time = time.time()
take_result = amounts_only.take(5)
take_time = time.time() - start_time
print(f"   Result: {[f'${x:.2f}' for x in take_result]}")
print(f"   Execution time: {take_time:.4f}s\n")

# Action 4: collect() - be careful with large datasets!
print("4. Executing collect() action (first 100 elements)...")
start_time = time.time()
# Limit collection to avoid memory issues
limited_collect = amounts_only.take(100)  # Safer than collect()
collect_time = time.time() - start_time
print(f"   Collected {len(limited_collect)} elements")
print(f"   Execution time: {collect_time:.4f}s")

**Exercise 2.1**: Compare execution times of different actions.

In [ ]:
# Solution: Execute different actions on category_amounts and measure timing
execution_times = {}

# Action 1: Count unique categories
print("Measuring action execution times...")

start_time = time.time()
unique_categories_count = category_amounts.keys().distinct().count()
execution_times['distinct_count'] = time.time() - start_time
print(f"Unique categories: {unique_categories_count} (Time: {execution_times['distinct_count']:.4f}s)")

# Action 2: Calculate total by category using reduceByKey + collect
start_time = time.time()
category_totals = category_amounts.reduceByKey(lambda a, b: a + b).collect()
execution_times['reduce_collect'] = time.time() - start_time
print(f"Category totals calculated (Time: {execution_times['reduce_collect']:.4f}s)")

# Action 3: Find maximum amount using reduce
start_time = time.time()
max_amount = category_amounts.values().reduce(lambda a, b: max(a, b))
execution_times['max_reduce'] = time.time() - start_time
print(f"Maximum amount: ${max_amount:.2f} (Time: {execution_times['max_reduce']:.4f}s)")

# Action 4: Sample data
start_time = time.time()
sample_data = category_amounts.sample(False, 0.1).count()
execution_times['sample_count'] = time.time() - start_time
print(f"Sample count: {sample_data} (Time: {execution_times['sample_count']:.4f}s)")

# Validation
assert unique_categories_count > 0, "Should have at least one category"
assert len(category_totals) > 0, "Should have category totals"
assert max_amount > 0, "Max amount should be positive"

print("\n✓ Exercise 2.1 completed successfully!")
print("\nExecution Time Summary:")
for action, time_taken in execution_times.items():
    print(f"  {action}: {time_taken:.4f}s")

### 2.2 Actions That Write Data

In [ ]:
# Actions that save data to external storage
import os
import tempfile

# Create temporary directory for outputs
temp_dir = tempfile.mkdtemp()
print(f"Using temporary directory: {temp_dir}")

# Save high-value transactions
output_path = os.path.join(temp_dir, "high_value_amounts")

print("\nSaving data with actions...")
start_time = time.time()

# This is an action that triggers execution and writes to disk
amounts_only.saveAsTextFile(output_path)

save_time = time.time() - start_time
print(f"Data saved in {save_time:.4f}s")

# Verify the save worked
saved_files = os.listdir(output_path)
print(f"Files created: {len([f for f in saved_files if f.startswith('part-')])} partition files")

# Clean up
import shutil
shutil.rmtree(temp_dir)
print("Temporary files cleaned up")

## Part 3: Performance Implications

Understanding when computation happens is crucial for optimization.

### 3.1 Multiple Actions on Same RDD

In [ ]:
# Demonstrate recomputation without caching
print("Testing recomputation behavior...\n")

# Create a complex RDD that takes time to compute
complex_rdd = parsed_transactions \
    .filter(lambda t: t['amount'] > 50) \
    .map(lambda t: (t['category'], t['amount'])) \
    .filter(lambda x: x[1] < 500)

print("🔄 Without caching - each action recomputes the entire chain:")

# First action
start_time = time.time()
count1 = complex_rdd.count()
time1 = time.time() - start_time
print(f"   First count(): {count1} records in {time1:.4f}s")

# Second action - recomputes everything!
start_time = time.time()
first_elem = complex_rdd.first()
time2 = time.time() - start_time
print(f"   First element: {first_elem} in {time2:.4f}s")

# Third action - recomputes again!
start_time = time.time()
sample_data = complex_rdd.take(10)
time3 = time.time() - start_time
print(f"   Sample (10 elements) retrieved in {time3:.4f}s")

total_time_no_cache = time1 + time2 + time3
print(f"\n📊 Total time without caching: {total_time_no_cache:.4f}s")

### 3.2 Caching for Performance

In [ ]:
# Now let's cache the RDD and see the difference
print("\n💾 With caching - computation happens once:")

# Cache the RDD
complex_rdd.cache()

# First action - computes and caches
start_time = time.time()
count1_cached = complex_rdd.count()
time1_cached = time.time() - start_time
print(f"   First count() (with caching): {count1_cached} in {time1_cached:.4f}s")

# Second action - uses cache!
start_time = time.time()
first_elem_cached = complex_rdd.first()
time2_cached = time.time() - start_time
print(f"   First element (from cache): {first_elem_cached} in {time2_cached:.4f}s")

# Third action - uses cache!
start_time = time.time()
sample_cached = complex_rdd.take(10)
time3_cached = time.time() - start_time
print(f"   Sample (from cache): retrieved in {time3_cached:.4f}s")

total_time_cached = time1_cached + time2_cached + time3_cached
print(f"\n📊 Total time with caching: {total_time_cached:.4f}s")
print(f"🚀 Performance improvement: {((total_time_no_cache - total_time_cached) / total_time_no_cache * 100):.1f}%")

# Check cache status
print(f"\n📋 Cache info:")
print(f"   Is cached: {complex_rdd.is_cached}")
print(f"   Storage level: {complex_rdd.getStorageLevel()}")

# Clean up cache
complex_rdd.unpersist()

**Exercise 3.1**: Implement your own caching performance test.

In [ ]:
# Solution: Create a performance test comparing cached vs uncached operations

# Step 1: Create a computationally expensive RDD
expensive_rdd = parsed_transactions \
    .filter(lambda t: t['amount'] > 25) \
    .map(lambda t: (t['customer_id'], t['amount'] * 1.1)) \
    .filter(lambda x: x[1] > 30)

print("🧪 Performance Test: Cached vs Uncached")
print("=" * 50)

# Test WITHOUT caching
print("\n📈 WITHOUT CACHING:")
uncached_times = []

for i in range(3):
    start_time = time.time()
    if i == 0:
        result = expensive_rdd.count()
    elif i == 1:
        result = expensive_rdd.take(5)
    else:
        result = expensive_rdd.map(lambda x: x[1]).max()
    
    execution_time = time.time() - start_time
    uncached_times.append(execution_time)
    print(f"   Operation {i+1}: {execution_time:.4f}s")

# Test WITH caching
print("\n💾 WITH CACHING:")
expensive_rdd.cache()
cached_times = []

for i in range(3):
    start_time = time.time()
    if i == 0:
        result = expensive_rdd.count()
    elif i == 1:
        result = expensive_rdd.take(5)
    else:
        result = expensive_rdd.map(lambda x: x[1]).max()
    
    execution_time = time.time() - start_time
    cached_times.append(execution_time)
    print(f"   Operation {i+1}: {execution_time:.4f}s")

# Calculate improvements
total_uncached = sum(uncached_times)
total_cached = sum(cached_times)
improvement = ((total_uncached - total_cached) / total_uncached) * 100

print(f"\n📊 RESULTS:")
print(f"   Total time without cache: {total_uncached:.4f}s")
print(f"   Total time with cache: {total_cached:.4f}s")
print(f"   Performance improvement: {improvement:.1f}%")

# Validation
assert expensive_rdd.is_cached, "RDD should be cached"
# Note: For small educational datasets, caching overhead may exceed benefits
# In production with larger datasets, caching typically provides significant improvements
if improvement > 0:
    print(f"🚀 Caching provided {improvement:.1f}% improvement!")
else:
    print(f"📚 Educational note: Small datasets may show caching overhead ({improvement:.1f}%)")
    print("   In production with larger datasets, caching typically provides significant benefits")

print("\n✓ Exercise 3.1 completed successfully!")

# Cleanup
expensive_rdd.unpersist()

## Part 4: Understanding Job Execution

Let's examine how Spark breaks down our operations into jobs, stages, and tasks.

### 4.1 Simple vs Complex Operations

In [ ]:
# Simple operation - narrow transformations only
print("🔍 Analyzing simple operation (narrow transformations):")
simple_rdd = parsed_transactions \
    .filter(lambda t: t['amount'] > 100) \
    .map(lambda t: t['amount'])

print(f"Lineage for simple operation:")
print(simple_rdd.toDebugString().decode('utf-8'))

# Execute and time
start_time = time.time()
simple_count = simple_rdd.count()
simple_time = time.time() - start_time
print(f"\nSimple operation result: {simple_count} records in {simple_time:.4f}s")

In [ ]:
# Complex operation - includes wide transformation (shuffle)
print("\n🔀 Analyzing complex operation (includes shuffle):")
complex_rdd = parsed_transactions \
    .map(lambda t: (t['category'], t['amount'])) \
    .reduceByKey(lambda a, b: a + b) \
    .sortBy(lambda x: x[1], ascending=False)

print(f"Lineage for complex operation:")
print(complex_rdd.toDebugString().decode('utf-8'))

# Execute and time
start_time = time.time()
complex_result = complex_rdd.collect()
complex_time = time.time() - start_time
print(f"\nComplex operation result: {len(complex_result)} categories in {complex_time:.4f}s")
print(f"Top 3 categories by total amount:")
for category, amount in complex_result[:3]:
    print(f"  {category}: ${amount:,.2f}")

**Exercise 4.1**: Compare narrow vs wide transformations.

In [ ]:
# Solution: Create and compare narrow vs wide transformation pipelines

print("🔬 Comparing Narrow vs Wide Transformations")
print("=" * 50)

# Narrow transformation pipeline (no shuffle)
print("\n📏 NARROW TRANSFORMATIONS (no data movement):")
narrow_pipeline = parsed_transactions \
    .filter(lambda t: t['category'] == 'Electronics') \
    .map(lambda t: t['amount'] * 2) \
    .filter(lambda amount: amount > 100)

# Time narrow operations
start_time = time.time()
narrow_count = narrow_pipeline.count()
narrow_max = narrow_pipeline.max()
narrow_time = time.time() - start_time

print(f"   Count: {narrow_count}")
print(f"   Max: ${narrow_max:.2f}")
print(f"   Execution time: {narrow_time:.4f}s")
print(f"   Partitions: {narrow_pipeline.getNumPartitions()}")

# Wide transformation pipeline (includes shuffle)
print("\n🌐 WIDE TRANSFORMATIONS (data shuffle required):")
wide_pipeline = parsed_transactions \
    .filter(lambda t: t['category'] == 'Electronics') \
    .map(lambda t: (t['customer_id'], t['amount'])) \
    .reduceByKey(lambda a, b: a + b) \
    .filter(lambda x: x[1] > 200) \
    .sortBy(lambda x: x[1], ascending=False)

# Time wide operations
start_time = time.time()
wide_count = wide_pipeline.count()
top_customer = wide_pipeline.first()
wide_time = time.time() - start_time

print(f"   High-spending customers: {wide_count}")
print(f"   Top customer: {top_customer[0]} with ${top_customer[1]:.2f}")
print(f"   Execution time: {wide_time:.4f}s")
print(f"   Partitions: {wide_pipeline.getNumPartitions()}")

# Analysis
print(f"\n📊 PERFORMANCE ANALYSIS:")
print(f"   Narrow transformations: {narrow_time:.4f}s")
print(f"   Wide transformations: {wide_time:.4f}s")
print(f"   Performance ratio: {wide_time/narrow_time:.1f}x slower")

# Show lineage complexity
print(f"\n🔍 LINEAGE COMPLEXITY:")
print(f"   Narrow pipeline stages: {len(narrow_pipeline.toDebugString().decode('utf-8').split('|'))}")
print(f"   Wide pipeline stages: {len(wide_pipeline.toDebugString().decode('utf-8').split('|'))}")

# Validation
assert narrow_count > 0, "Should have narrow transformation results"
assert wide_count > 0, "Should have wide transformation results"
# Note: With small educational datasets, timing differences may be negligible or reversed due to overhead
# In production with larger datasets, wide transformations typically take longer due to shuffle operations
assert wide_time >= 0 and narrow_time >= 0, "Both operations should complete successfully"

print("\n✓ Exercise 4.1 completed successfully!")

## Part 5: Common Patterns and Best Practices

Learn to identify and optimize common transformation/action patterns.

### 5.1 Avoiding Repeated Actions

In [ ]:
# Bad pattern: Multiple actions without caching
print("❌ BAD PATTERN: Multiple actions without caching")

def bad_analysis_pattern():
    # This RDD will be recomputed for each action!
    analysis_rdd = parsed_transactions \
        .filter(lambda t: t['amount'] > 50) \
        .map(lambda t: (t['category'], t['amount']))
    
    start_time = time.time()
    
    # Each of these actions recomputes the entire chain
    total_records = analysis_rdd.count()
    category_totals = analysis_rdd.reduceByKey(lambda a, b: a + b).collect()
    max_transaction = analysis_rdd.map(lambda x: x[1]).max()
    sample_data = analysis_rdd.take(10)
    
    bad_time = time.time() - start_time
    return bad_time, total_records, len(category_totals)

bad_time, records, categories = bad_analysis_pattern()
print(f"   Time: {bad_time:.4f}s, Records: {records}, Categories: {categories}")

print("\n✅ GOOD PATTERN: Cache before multiple actions")

def good_analysis_pattern():
    # Cache the RDD since we'll use it multiple times
    analysis_rdd = parsed_transactions \
        .filter(lambda t: t['amount'] > 50) \
        .map(lambda t: (t['category'], t['amount'])) \
        .cache()  # Cache here!
    
    start_time = time.time()
    
    # First action computes and caches, others use cache
    total_records = analysis_rdd.count()
    category_totals = analysis_rdd.reduceByKey(lambda a, b: a + b).collect()
    max_transaction = analysis_rdd.map(lambda x: x[1]).max()
    sample_data = analysis_rdd.take(10)
    
    good_time = time.time() - start_time
    
    # Clean up
    analysis_rdd.unpersist()
    
    return good_time, total_records, len(category_totals)

good_time, records, categories = good_analysis_pattern()
print(f"   Time: {good_time:.4f}s, Records: {records}, Categories: {categories}")

improvement = ((bad_time - good_time) / bad_time) * 100
print(f"\n🚀 Caching improvement: {improvement:.1f}% faster")

### 5.2 Optimizing Transformation Order

In [ ]:
# Demonstrate the importance of filter ordering
print("🔧 Optimizing transformation order")

# Inefficient: expensive operations before filtering
print("\n❌ INEFFICIENT: Complex operations before filtering")
start_time = time.time()
inefficient = parsed_transactions \
    .map(lambda t: {
        **t,
        'tax': t['amount'] * 0.08,
        'total_with_tax': t['amount'] * 1.08,
        'category_upper': t['category'].upper()
    }) \
    .filter(lambda t: t['amount'] > 200) \
    .filter(lambda t: t['category'] == 'Electronics')

inefficient_result = inefficient.count()
inefficient_time = time.time() - start_time
print(f"   Result: {inefficient_result} records in {inefficient_time:.4f}s")

# Efficient: filter first, then expensive operations
print("\n✅ EFFICIENT: Filter first, then complex operations")
start_time = time.time()
efficient = parsed_transactions \
    .filter(lambda t: t['category'] == 'Electronics') \
    .filter(lambda t: t['amount'] > 200) \
    .map(lambda t: {
        **t,
        'tax': t['amount'] * 0.08,
        'total_with_tax': t['amount'] * 1.08,
        'category_upper': t['category'].upper()
    })

efficient_result = efficient.count()
efficient_time = time.time() - start_time
print(f"   Result: {efficient_result} records in {efficient_time:.4f}s")

optimization = ((inefficient_time - efficient_time) / inefficient_time) * 100
print(f"\n🎯 Optimization gain: {optimization:.1f}% faster by filtering early")

assert inefficient_result == efficient_result, "Both approaches should produce same results"

**Exercise 5.1**: Implement your own optimization comparison.

In [ ]:
# Solution: Create two versions of the same analysis - unoptimized vs optimized

print("🎯 Optimization Challenge: Customer Spending Analysis")
print("=" * 55)

# UNOPTIMIZED VERSION
print("\n❌ UNOPTIMIZED VERSION:")
print("   - Complex transformations first")
print("   - No caching despite multiple actions")
print("   - Wide transformations before narrow filtering")

def unoptimized_analysis():
    start_time = time.time()
    
    # Bad: Complex operations first, then filtering
    customer_analysis = parsed_transactions \
        .map(lambda t: (t['customer_id'], {
            'amount': t['amount'],
            'category': t['category'],
            'tax_rate': 0.08 if t['category'] == 'Electronics' else 0.06,
            'total_with_tax': t['amount'] * (1.08 if t['category'] == 'Electronics' else 1.06)
        })) \
        .groupByKey() \
        .mapValues(lambda transactions: {
            'total_spent': sum(t['total_with_tax'] for t in transactions),
            'transaction_count': len(list(transactions))
        }) \
        .filter(lambda x: x[1]['total_spent'] > 500) \
        .filter(lambda x: x[1]['transaction_count'] > 2)
    
    # Multiple actions without caching
    high_spender_count = customer_analysis.count()
    top_spender = customer_analysis.max(key=lambda x: x[1]['total_spent'])
    avg_spending = customer_analysis.map(lambda x: x[1]['total_spent']).mean()
    
    unopt_time = time.time() - start_time
    return unopt_time, high_spender_count, top_spender, avg_spending

unopt_time, unopt_count, unopt_top, unopt_avg = unoptimized_analysis()
print(f"   Execution time: {unopt_time:.4f}s")
print(f"   Results: {unopt_count} customers, Top: ${unopt_top[1]['total_spent']:.2f}, Avg: ${unopt_avg:.2f}")

# OPTIMIZED VERSION
print("\n✅ OPTIMIZED VERSION:")
print("   - Same algorithm with caching for multiple actions")
print("   - Identical filtering and transformation logic")

def optimized_analysis():
    start_time = time.time()
    
    # Good: Same algorithm but with caching for multiple actions
    customer_analysis = parsed_transactions \
        .map(lambda t: (t['customer_id'], {
            'amount': t['amount'],
            'category': t['category'],
            'tax_rate': 0.08 if t['category'] == 'Electronics' else 0.06,
            'total_with_tax': t['amount'] * (1.08 if t['category'] == 'Electronics' else 1.06)
        })) \
        .groupByKey() \
        .mapValues(lambda transactions: {
            'total_spent': sum(t['total_with_tax'] for t in transactions),
            'transaction_count': len(list(transactions))
        }) \
        .filter(lambda x: x[1]['total_spent'] > 500) \
        .filter(lambda x: x[1]['transaction_count'] > 2) \
        .cache()  # Cache here for multiple actions!
    
    # Multiple actions using cache
    high_spender_count = customer_analysis.count()
    top_spender = customer_analysis.max(key=lambda x: x[1]['total_spent'])
    avg_spending = customer_analysis.map(lambda x: x[1]['total_spent']).mean()
    
    opt_time = time.time() - start_time
    
    # Clean up
    customer_analysis.unpersist()
    
    return opt_time, high_spender_count, top_spender, avg_spending

opt_time, opt_count, opt_top, opt_avg = optimized_analysis()
print(f"   Execution time: {opt_time:.4f}s")
print(f"   Results: {opt_count} customers, Top: ${opt_top[1]['total_spent']:.2f}, Avg: ${opt_avg:.2f}")

# Performance comparison
speedup = unopt_time / opt_time
improvement = ((unopt_time - opt_time) / unopt_time) * 100

print(f"\n🚀 OPTIMIZATION RESULTS:")
print(f"   Unoptimized time: {unopt_time:.4f}s")
print(f"   Optimized time: {opt_time:.4f}s")
print(f"   Speedup: {speedup:.1f}x faster")
print(f"   Improvement: {improvement:.1f}%")

# Validation
# Note: For small datasets, optimization overhead may sometimes exceed benefits
# The key learning is understanding the optimization patterns
if opt_time < unopt_time:
    print("🚀 Optimized version was faster as expected!")
else:
    print("📚 Educational note: Small datasets may not show optimization benefits due to overhead")
    print("   In production with larger datasets, these optimizations provide significant improvements")

# Results should be exactly the same since we use identical algorithms, just with/without caching
result_difference = abs(unopt_count - opt_count)
assert result_difference == 0, f"Results should be identical since same algorithm is used: difference of {result_difference}"

print("\n✓ Exercise 5.1 completed successfully!")

## Summary and Key Takeaways

Congratulations! You've mastered the distinction between transformations and actions in Spark.

### 🎯 Key Concepts Mastered:

1. **Lazy Evaluation**: Transformations are lazy - they define computation graphs without executing
2. **Eager Execution**: Actions trigger immediate execution of the entire transformation chain
3. **Performance Impact**: Multiple actions without caching cause recomputation
4. **Optimization Strategies**: 
   - Cache RDDs used by multiple actions
   - Filter early to reduce data size
   - Minimize shuffle operations
   - Understand narrow vs wide transformations

### 📊 Transformations vs Actions Quick Reference:

| **Transformations (Lazy)** | **Actions (Eager)** |
|----------------------------|----------------------|
| `map()`, `filter()`, `flatMap()` | `count()`, `collect()`, `first()` |
| `reduceByKey()`, `groupByKey()` | `take()`, `reduce()`, `max()` |
| `join()`, `union()`, `distinct()` | `saveAsTextFile()`, `foreach()` |
| Return new RDD/DataFrame | Return values or write data |
| Build execution graph | Trigger computation |

### 🚀 Performance Best Practices:

- **Cache strategically**: Use `.cache()` for RDDs accessed multiple times
- **Filter early**: Reduce data size before expensive operations
- **Avoid collect()**: Use `take()` or `sample()` for large datasets
- **Monitor Spark UI**: Understand job execution and identify bottlenecks

In [ ]:
# Final cleanup
spark.stop()
print("\n🎉 Lab 2 completed successfully!")
print("📈 You now understand the power of lazy evaluation in Spark!")
print("\n➡️  Next: Lab 3 - Lazy Evaluation Deep Dive")